<a href="https://colab.research.google.com/github/Naraenbaskaran18/Ridge-Regression/blob/main/Boston_Week_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [48]:
class RidgeRegression:
    def __init__(self, alpha=1):
        self.alpha = alpha
        self.theta = None
        
    def cost_function(self, X, y, theta):
        m = len(y) 
        y_pred = X.dot(theta)
        cost = (1/(2*m)) * np.sum((y_pred - y)**2) + (self.alpha/(2*m)) * np.sum(theta[1:]**2)
        return cost
    
    def gradient_function(self, X, y, theta):
        m = len(y)
        y_pred = X.dot(theta)
        grad = (1/m) * (X.T.dot(y_pred - y)) + (self.alpha/m) * np.concatenate(([0], theta[1:]))
        return grad
    
    def fit(self, X, y, learning_rate=0.01, num_iterations=1000):
        m, n = X.shape
        X = np.concatenate((np.ones((m, 1)), X), axis=1)
        self.theta = np.zeros(n+1)
        
        for i in range(num_iterations):
            grad = self.gradient_function(X, y, self.theta)
            self.theta = self.theta - learning_rate * grad
            
    def predict(self, X):
        m = X.shape[0]
        X = np.concatenate((np.ones((m, 1)), X), axis=1)
        return X.dot(self.theta)
    
    def grid_search(self, X_train, y_train, X_test, y_test, alpha_values, learning_rate_values, num_iterations_values):
        best_alpha = None
        best_learning_rate = None
        best_num_iterations = None
        best_r2 = -np.inf
        
        for alpha in alpha_values:
            for learning_rate in learning_rate_values:
                for num_iterations in num_iterations_values:
                    self.alpha = alpha
                    self.fit(X_train, y_train, learning_rate=learning_rate, num_iterations=num_iterations)
                    y_pred = self.predict(X_test)
                    r2 = r2_score(y_test, y_pred)
                    if r2 > best_r2:
                        best_r2 = r2
                        best_alpha = alpha
                        best_learning_rate = learning_rate
                        best_num_iterations = num_iterations
        
        self.alpha = best_alpha
        self.fit(X_train, y_train, learning_rate=best_learning_rate, num_iterations=best_num_iterations)
        print("Best hyperparameters:")
        print("alpha:", best_alpha)
        print("learning rate:", best_learning_rate)
        print("num iterations:", best_num_iterations)
        print("best R2 score:", best_r2)

In [49]:
# load the Boston Housing dataset
boston = fetch_openml(name='boston')
X, y = boston.data, boston.target

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# split the dataset into training and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



/usr/local/lib/python3.8/dist-packages/sklearn/datasets/_openml.py:292: UserWarning: Multiple active versions of the dataset matching the name boston exist. Versions may be fundamentally different, returning version 1.
  warn(
/usr/local/lib/python3.8/dist-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [50]:
# create an instance of the RidgeRegression class
ridge_reg = RidgeRegression()



In [51]:
# Define the hyperparameters to search over
alpha_values = [0.01, 0.1, 1, 10]
learning_rate_values = [0.001, 0.01,0.1]
num_iterations_values = [100,1000,10000]

In [52]:
#Perform grid search to find the best hyperparameters
ridge_reg.grid_search(X_train, y_train, X_test, y_test, alpha_values, learning_rate_values, num_iterations_values)

Best hyperparameters:
alpha: 0.01
learning rate: 0.1
num iterations: 10000
best R2 score: 0.6687562987931833


In [43]:
# make predictions on the test data
y_pred = ridge_reg.predict(X_test)

# compute the R2 score
r2 = r2_score(y_test, y_pred)
print("R2 score on test set:", r2)

R2 score on test set: 0.6687591740646741


In [46]:
# compute the mse
mse = mean_squared_error(y_test, y_pred)
print("mse:", mse)

mse: 24.291142902987474
